In [384]:
# import requests and beautiful soup for parsing
# run 'pip install requests' or 'pip install bs4' if needed
import requests
import bs4
import json
import pandas
import re

# import required modules and set up environment
import os

# replace file path below with your own local convokit
os.chdir('/Users/marianneaubin/Documents/Classes/CS6742/cs6742-fork')

import convokit
from convokit import Corpus, User, Utterance, Parser
from bs4 import NavigableString

In [371]:
def is_last_page(menu):
    counter = menu.text.replace('Results ', '').replace('to ','').replace('of ','')
    comment_counts = counter.split(' ')
    comment_counts = [int(i) for i in comment_counts] 
    if comment_counts[1] == comment_counts[2]:
        return True
    return False

In [463]:
def get_all_urls():
    titles = []
    urls = []
    url = 'https://www.debatepolitics.com/gun-control/'
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.text,'lxml')
    titles_info = soup.find_all("a", class_="title")
    for thread in titles_info:
        url = thread["href"].split('.html')
        urls.append(url[0])
        titles.append(thread.text)
    return titles, urls

In [464]:
titles, urls = get_all_urls()
print(titles, urls)

['Non-partisan PSA: machine guns, full-auto, semi-auto, assault weapons...', 'Are Gun banners anti woman?', 'Gun in Home Kills Loved ones and Owner', "SCOTUS Allos Sandy Hook Families' Case Against Remington Arms to Proceed", 'Constitutional carry now in effect in Oklahoma.', 'Inherent risk for criminals', 'Wrong career choice.', 'You Guys Can Exhale Now!', "Republicans aren't wrong about 'People with Metnal health Issues'", 'Man making illegal rounds used in Vegas pleads guilty', '10 yr old shot two weeks after 2 yr old', 'Crossroads of the West: inside the last gun show in San Francisco', 'YAY!! Walmart steps up!', 'CA school shooting prevented', 'Gun Control Hypocrisy', 'Virginians voted their rights away', "...you're More Likely to Shoot a Loved One", 'Minority Rights', 'NRA reportedly made unapproved big-money deals with celebrity board members', 'NRA turmoil creates rift among some big donors', 'The number of concealed handgun permits has increased for the third year in a row.'] 

In [406]:
# this dictionary is the uber dictionary
# each thread title is a key 

users = {}
comments = []

page_num = 1
cont = True
while cont:
    url = 'https://www.debatepolitics.com/gun-control/375772-inherent-risk-criminals' + str(-page_num) + '.html'
    resp = requests.get(url)
    soup = bs4.BeautifulSoup(resp.text,'lxml')
    if(is_last_page(soup.find("div", class_="postpagestats"))):
        cont = False
    page_num = page_num + 1
    users, comments = get_metadata(soup, users, comments)
print(users)
print(comments)

{'Bum': {'join_date': 'Aug 2017', 'gender': 'Male', 'lean': 'Other ', 'posts': 5052}, 'TheParser': {'join_date': 'Jul 2018', 'gender': 'Male', 'lean': 'Moderate ', 'posts': 3626}, 'Rich2018': {'join_date': 'Mar 2018', 'gender': 'Male', 'lean': 'Liberal ', 'posts': 10421}, 'TurtleDude': {'join_date': 'Oct 2005', 'gender': 'Male', 'lean': 'Libertarian - Right ', 'posts': 221818}, 'AProudLefty': {'join_date': 'Jul 2019', 'gender': 'Male', 'lean': 'Liberal ', 'posts': 7060}, 'Lursa': {'join_date': 'May 2013', 'gender': 'Female', 'lean': 'Independent ', 'posts': 53702}, 'Court Jester': {'join_date': 'Nov 2019', 'gender': 'Male', 'lean': 'Independent ', 'posts': 181}, 'RetiredUSN': {'join_date': 'Jan 2016', 'gender': 'Male', 'lean': 'Other ', 'posts': 15997}, 'KevinKohler': {'join_date': 'Oct 2011', 'gender': 'Male', 'lean': 'Undisclosed ', 'posts': 20620}, 'dogboy49': {'join_date': 'Dec 2017', 'lean': 'Libertarian ', 'posts': 312}, 'apdst': {'join_date': 'Jun 2009', 'gender': 'Male', 'lean'

In [378]:
def get_usernames(soup):
    names = []
    names_raw = soup.find_all("a", class_="username online popupctrl")
    names_raw = names_raw + soup.find_all("a", class_="username offline popupctrl")
    for name_raw in names_raw:
        names.append(name_raw.strong.text)
    return names

print(get_usernames(soup))

['ttwtt78640', 'Lursa', 'TurtleDude', 'Bodhisattva', 'TurtleDude', 'Bodhisattva', 'TurtleDude', 'JMR', 'Lursa', 'JMR']


In [379]:
def extract_username(postinfo):
    name_raw = postinfo.find("a", class_="username offline popupctrl")
    if name_raw is None:
        name_raw = postinfo.find("a", class_="username online popupctrl")
    name = name_raw.strong.text
    return name

In [380]:
def extract_metadata(postinfo):
    jd_raw = postinfo.find("dl", class_="userinfo_extra")
    
    meta = {}
    meta_i = {}
    for i,jd in enumerate(jd_raw):
        if not isinstance(jd, NavigableString):     
            if jd.text == "Join Date":
                meta_i[i+2] = "join_date"
            elif jd.text == "Gender":
                meta_i[i+2] = "gender"
            elif jd.text == "Lean":
                meta_i[i+2] = "lean"
            elif jd.text == "Posts":
                meta_i[i+2] = "posts"
    for j, jd in enumerate(jd_raw):
        if j in meta_i.keys():
            meta[meta_i[j]] = jd
            
    for elem in meta.keys():
        if elem == "join_date":
            meta[elem] = meta[elem].text
        elif elem == "gender":
            meta[elem] = (meta[elem].find("img")["src"]).replace('https://www.debatepolitics.com/images/misc/','').\
                replace('.gif','')
            if meta[elem] == '':
                meta[elem] = "Unknown"
        elif elem == "lean":
            meta[elem] = meta[elem].text
        elif elem == "posts":
            meta[elem] = int(meta[elem].text.replace(',',''))
    
    return meta

In [381]:
def extract_comment(postinfo):
    postedby = postinfo.find("div", class_="bbcode_postedby")
    message = postinfo.find("div", class_="message")
    comment = postinfo.find("blockquote", class_="postcontent restore")
    links = comment.find("a")
    if links != None:
        links.decompose()
    if message != None:
        message.decompose()
    if postedby != None:
        postedby.decompose()
    comment = str(comment.text)
    comment = re.sub('\s\s+', ' ', comment).strip(' ')
    return comment

In [400]:
#import NavigableString

def get_metadata(soup, users_dict, comments_list):
    # meta dict will have a key for each user name
    # the remaining attributes will be in the dict
    # attributes are "join_date", "gender", "lean", "posts", "location", "comment"
    postinfo_list = soup.find_all("div", class_="postdetails")
    
    #for each user
    for postinfo in postinfo_list:
        #extract name of user
        username = extract_username(postinfo)
        #extract metadata
        if username not in users_dict.keys():
            metadata = extract_metadata(postinfo)
            users_dict[username] = {}
            for elem in metadata.keys():
                users_dict[username][elem] = metadata[elem]
        comment = extract_comment(postinfo)
        comments_list.append((username, comment))
    return(users_dict, comments_list)

In [93]:
soup.find_all("div", class_="postdetails")

[<div class="postdetails">
 <div class="userinfo">
 <div class="username_container">
 <div class="popupmenu memberaction">
 <a class="username online popupctrl" href="https://www.debatepolitics.com/members/jacksinpa.html?s=42f4be4a6a237d77488c2302a4249a8f" title="JacksinPA is online now"><strong>JacksinPA</strong></a>
 <ul class="popupbody popuphover memberaction_body">
 <li class="left">
 <a class="siteicon_profile" href="https://www.debatepolitics.com/members/jacksinpa.html?s=42f4be4a6a237d77488c2302a4249a8f">
 				View Profile
 			</a>
 </li>
 <li class="right">
 <a class="siteicon_forum" href="https://www.debatepolitics.com/search.php?do=finduser&amp;userid=32026&amp;contenttype=vBForum_Post&amp;showposts=1&amp;s=42f4be4a6a237d77488c2302a4249a8f" rel="nofollow">
 				View Forum Posts
 			</a>
 </li>
 <li class="left">
 <a class="siteicon_message" href="https://www.debatepolitics.com/private.php?do=newpm&amp;u=32026&amp;s=42f4be4a6a237d77488c2302a4249a8f" rel="nofollow">
 				Privat

In [29]:
print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html dir="ltr" id="vbulletin_html" lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <base href="https://www.debatepolitics.com/"/>
  <!--[if IE]></base><![endif]-->
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="https://www.debatepolitics.com" id="e_vb_meta_bburl" name="vb_meta_bburl"/>
  <meta content="vBulletin 4.2.3" name="generator"/>
  <meta content="IE=9" http-equiv="X-UA-Compatible"/>
  <link href="https://www.debatepolitics.com/favicon.ico" rel="Shortcut Icon" type="image/x-icon"/>
  <script src="https://ajax.googleapis.com/ajax/libs/yui/2.9.0/build/yuiloader-dom-event/yuiloader-dom-event.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   <!--
	if (typeof YAHOO === 'undefined') // Load ALL YUI Local
	{
		document.write('<script type="text/javascript" src="clientscript/yui/yuiloader-

In [250]:
#soup.title.string
#for tag in soup.find_all(True):
    print(tag.name)

html
head
base
meta
meta
meta
meta
link
script
script
script
link
link
link
link
script
script
link
link
link
link
meta
meta
title
link
link
link
script
body
div
div
div
a
img
div
ul
li
a
li
a
li
script
form
fieldset
div
div
input
input
input
input
div
label
input
input
input
input
input
input
script
div
hr
div
ul
li
a
ul
li
a
li
a
li
a
li
a
ul
li
a
li
a
li
a
li
a
ul
li
a
li
a
ul
li
a
li
a
li
a
span
li
a
li
a
div
form
input
input
span
span
input
span
span
input
ul
li
a
div
div
ul
li
a
img
li
a
span
li
a
span
li
a
span
li
a
span
li
span
hr
form
input
input
input
input
input
ol
li
b
b
b
a
b
a
b
div
div
form
input
input
span
a
span
a
span
a
span
a
span
a
span
a
span
span
a
img
span
a
img
ul
li
label
input
input
div
div
h1
span
a
div
div
ul
li
h6
a
ul
li
a
li
li
h6
a
ul
li
label
li
a
li
a
div
ol
li
div
span
span
span
span
a
a
div
div
div
div
a
strong
ul
li
a
li
a
li
a
li
a
img
dd
i
span
span
img
br
img
br
img
br
img
hr
dl
dt
dd
dt
dd
dt
dd
dt
dd
div
div
a
img
a
img
div
div
div
h2
div
div
b